In [ ]:
import numpy as np, pandas as pd
from scipy.sparse import coo_matrix
from poismf import PoisMF
from pandas.api.types import CategoricalDtype
from scipy import sparse
from sklearn.model_selection import train_test_split
import tqdm
from recsys_metrics import *
import torch
# https://github.com/zuoxingdong/recsys_metrics?tab=readme-ov-file#Citation


In [2]:
train_df = pd.read_csv("pf_last_lastfm_train.tsv",sep="\t",    names=[
        'UserId', 'ItemId', 'Count', 'timestamp'
    ],)
test_df = pd.read_csv("pf_last_lastfm_test.tsv",sep="\t",    names=[
        'UserId', 'ItemId', 'Count', 'timestamp'
    ],)


In [3]:
display(test_df)
display(train_df)

,UserId,ItemId,Count,timestamp
0,0,638469,1,1208792880
1,1,357980,1,1190979091
2,2,309977,1,1191092348
3,3,49137,1,1219791510
4,4,503730,1,1179403857
...,...,...,...,...
5080835,821,835160,1,1183327967
5080836,719,639274,1,1191020924
5080837,8,107684,1,1175035475
5080838,158,709274,1,1216208176


,UserId,ItemId,Count,timestamp
0,0,929335,1,1241442369
1,0,267920,1,1241441734
2,0,41330,1,1241363693
3,0,294494,1,1241363418
4,0,942636,1,1241363071
...,...,...,...,...
11855289,991,665723,1,1201472079
11855290,991,99910,1,1201471828
11855291,991,616930,1,1201471355
11855292,991,235234,1,1201471012


In [4]:
test_users =test_df["UserId"].unique()
test_userDict ={}
for i in test_users:
    test_userDict[i] ={}

In [5]:
train_users =train_df["UserId"].unique()
train_userDict ={}
for i in train_users:
    train_userDict[i] ={}

In [16]:
for index, row in test_df.iterrows():
    user = row["UserId"]
    item = row["ItemId"]
    rating = row["Count"]
    if item in test_userDict[user]:
        test_userDict[user][item]+=1
    else:
        test_userDict[user][item]=1

In [17]:
for index, row in train_df.iterrows():
    user = row["UserId"]
    item = row["ItemId"]
    rating = row["Count"]
    if item in train_userDict[user]:
        train_userDict[user][item]+=1
    else:
        train_userDict[user][item]=1

In [18]:
train_res = []
for user, items in train_userDict.items():
    for item, rating in items.items():
        train_res.append((user, item, rating))

train_df = pd.DataFrame(train_res, columns=["UserId", "ItemId", "Count"])

In [19]:
test_res = []
for user, items in test_userDict.items():
    for item, rating in items.items():
        test_res.append((user, item, rating))

test_df = pd.DataFrame(test_res, columns=["UserId", "ItemId", "Count"])

In [29]:
display(train_df)
display(test_df)

,UserId,ItemId,Count
0,0,929335,23
1,0,267920,28
2,0,41330,4
3,0,294494,5
4,0,942636,8
...,...,...,...
3100554,991,849399,1
3100555,991,920272,1
3100556,991,902961,1
3100557,991,441089,1


,UserId,ItemId,Count
0,0,638469,3
1,0,521,1
2,0,549,1
3,0,551,1
5,0,3300,1
...,...,...,...
2377971,991,338462,1
2377972,991,807771,1
2377973,991,600214,1
2377974,991,290681,1


In [24]:
user_positives = test_df.groupby('UserId')['ItemId'].apply(set).to_dict()


In [25]:
test_users = test_df['UserId'].unique()
test_items = test_df['ItemId'].unique()


train_users = train_df['UserId'].unique()
train_items = train_df['ItemId'].unique()


user = list(set(train_users) & set(test_users))
item = list(set(train_items) & set(test_items))

In [26]:
train_df = train_df[train_df['UserId'].isin(user)]
train_df = train_df[train_df['ItemId'].isin(item)]

test_df = test_df[test_df['UserId'].isin(user)]
test_df = test_df[test_df['ItemId'].isin(item)]

In [27]:

test_ratings_matrix = test_df.pivot(index='UserId', columns='ItemId', values='Count')
train_ratings_matrix = train_df.pivot(index='UserId', columns='ItemId', values='Count')


In [28]:
display(train_ratings_matrix)
display(test_ratings_matrix)

ItemId,0,1,3,5,6,7,9,10,11,12,...,960387,960389,960392,960393,960394,960395,960396,960397,960398,960400
UserId,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ItemId,0,1,3,5,6,7,9,10,11,12,...,960387,960389,960392,960393,960394,960395,960396,960397,960398,960400
UserId,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
modelPF = PoisMF(reindex=True, method="tncg", use_float=True,
                    early_stop=False, reuse_prev=True,
                    k=20, niter=10, maxupd=200, l2_reg=1e3)\
                .fit(train_df)

In [42]:
del train_df
del test_df
del train_ratings_matrix
del user
del item
del test_res
del train_res
del test_userDict
del train_userDict
del test_users
del train_users


In [31]:
display(modelPF.A.shape) # user
display(modelPF.B.shape) # items


(991, 20)

(656942, 20)

In [32]:
pred = np.matmul(modelPF.A,modelPF.B.T)

In [34]:
display(pred.shape)

(991, 656942)

In [36]:
test_ratings_matrix[test_ratings_matrix >1] = 1

true = torch.tensor(test_ratings_matrix.fillna(0).to_numpy())


In [37]:
predv = torch.tensor(pred)


In [43]:
del modelPF
del pred
del test_ratings_matrix

In [38]:
display(true)
display(predv)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

tensor([[0.1493, 0.1532, 0.0483,  ..., 0.0000, 0.0000, 0.0000],
        [0.0031, 0.0000, 0.0000,  ..., 0.0005, 0.0005, 0.0005],
        [0.0015, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0058, 0.0000, 0.0000,  ..., 0.0086, 0.0086, 0.0086],
        [0.0025, 0.0000, 0.0000,  ..., 0.0046, 0.0046, 0.0046],
        [0.0005, 0.0000, 0.0000,  ..., 0.0099, 0.0099, 0.0099]])

In [39]:
def calculate_metrics(pred_df, user_postivies, k_list = [10,50,100]):
  metrics = {
    'MAR@ALL': [],
    'MAR@1000': [],
  }


  grouped = pred_df.groupby('user')

  for user, user_data in tqdm.tqdm(grouped, desc="Processing Users"):
    positives = user_postivies.get(user, set())
    if len(positives) == 0:
      continue

    sorted_data = user_data.sort_values(by='score', ascending=False)
    all_items = sorted_data['item'].tolist()

    relevant_ranks1k = []
    relevant_ranks_all=[]
    for rank, item in enumerate(all_items[:1000], start=1):
      if item in positives:
        relevant_ranks1k.append(rank)
    #calculate relevant
    for rank, item in enumerate(all_items, start=1):
      if item in positives:
        relevant_ranks_all.append(rank)



    if relevant_ranks_all:
      metrics['MAR@ALL'].append(np.mean(relevant_ranks_all))
    if relevant_ranks1k:
      metrics['MAR@1000'].append(np.mean(relevant_ranks1k))
      

  
  final = {
    'MAR@ALL': np.mean(metrics['MAR@ALL']),
    'MAR@1000': np.mean(metrics['MAR@1000']),
  }

  return final

In [40]:
def metricStuff(pred,true,user_positive):

    print("NDCG@10",normalized_dcg(pred, true ,k=10))
    print("MRR@1000",mean_reciprocal_rank(pred, true,k=1000))
    print("MRR@all",mean_reciprocal_rank(pred, true))
    marData = pred.detach().numpy()
    marDF = pd.DataFrame(marData)
    marDF.index.name = "user"
    marDF.columns.name = "items"
    marDF = marDF.stack().reset_index()
    marDF.columns = ['user', 'item', 'score']
    print("MAR",calculate_metrics(marDF,user_positive))
    print("MAP",mean_average_precision(pred, true))
    print("P@50",precision(pred, true,k=50))
    print("R@50",recall(pred, true,k=50))
    

In [41]:
metricStuff(predv,true,user_positives)

NDCG@10 tensor(0.0033)
MRR@1000 tensor(0.0175)
MRR@all tensor(0.0176)


Processing Users: 100%|██████████| 991/991 [03:51<00:00,  4.28it/s] 


MAR {'MAR@ALL': np.float64(328669.6823422113), 'MAR@1000': np.float64(512.6435314083727)}
MAP tensor(0.0032)
P@50 tensor(0.0033)
R@50 tensor(6.8030e-05)
